In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import csv
import os
import time


In [2]:
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect():
    engine = create_engine("mysql+pymysql://root:"
                +"big15"
                +"@localhost:3306/big15?charset=utf8"
                , encoding='utf8')
    
    return engine.connect()

In [3]:
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect_ip(ip_address, db_name):
    engine = create_engine("mysql+pymysql://admin:"
                +"big15" # user password
                +"@{0}:3306/{1}?charset=utf8".format(ip_address, db_name)
                , encoding='utf8')
    
    return engine.connect()

In [4]:
# Data csv => SQL에 추가
def sqlalchemy_data_insert(sqlalchemy_conn):
    
    # for문 돌려서 파일 리스트 받고 하나씩 읽는다
    for i, file in enumerate(os.listdir('../data/20221121/complete/')):
        stock_code_name = file[1:].split('.')[0]

        stock_info = pd.read_csv('../data/20221121/complete/{0}'.format(file))

        stock_info['년'] = stock_info['날짜'].apply(str).str[:4].astype('int')
        stock_info['월'] = stock_info['날짜'].apply(str).str[4:6].astype('int')
        stock_info['일'] = stock_info['날짜'].apply(str).str[6:8].astype('int')
        stock_info.sort_index(ascending=False, inplace=True)
        # name= table명
        # con= connect할 때 들어가는 아이디 비밀번호 등등
        # if_exists= 'append' : 기존 테이블이 있는 경우 데이터를 추가
        # 'fail' : 기존 테이블이 있는 경우 아무일도 없지만 없을 경우 valuesError뜸
        # 'replace' : 기존 테이블이 있을 경우, 기존 테이블을 삭제하고 다시 테이블을
        try:
            count = stock_info.to_sql(name='{0}'.format(stock_code_name), con=sqlalchemy_conn, if_exists='replace', index=False)
            stock_info = pd.DataFrame()
            print('처리된 row 수 : {0}, 순서 : {1}, 파일명 : {2}'.format(count, i+1, file))
        except:
            print('error 발생. 순서 : {0}, 파일명 {1}'.format(i+1, file))
        

        

        #     # 현재 DB 내 존재하는 테이블(종목) 추출
        # sql = "ALTER TABLE stock_info.{0} ADD PRIMARY KEY (날짜, 시간)".format(stock_code_name)

        # with pymysql_conn:
        #     with pymysql_conn.cursor() as cur:
        #         cur.execute(sql)
            
def get_pymysql_connection(ip_address, db_name):

    conn = pymysql.connect(host=ip_address, user='admin', password='big15'
                        , db=db_name, charset='utf8')

    return conn


In [7]:
sqlalchemy_data_insert(sqlalchemy_connect_ip('192.168.50.123', 'stock_info'))

처리된 row 수 : 194571, 순서 : 1, 파일명 : A000020.csv
처리된 row 수 : 194571, 순서 : 2, 파일명 : A000040.csv
처리된 row 수 : 194571, 순서 : 3, 파일명 : A000050.csv
처리된 row 수 : 194571, 순서 : 4, 파일명 : A000060.csv
처리된 row 수 : 194571, 순서 : 5, 파일명 : A000070.csv
처리된 row 수 : 194571, 순서 : 6, 파일명 : A000080.csv
처리된 row 수 : 194571, 순서 : 7, 파일명 : A000100.csv
처리된 row 수 : 194571, 순서 : 8, 파일명 : A000120.csv
처리된 row 수 : 194571, 순서 : 9, 파일명 : A000140.csv
처리된 row 수 : 194571, 순서 : 10, 파일명 : A000150.csv
처리된 row 수 : 194571, 순서 : 11, 파일명 : A000180.csv
처리된 row 수 : 194571, 순서 : 12, 파일명 : A000210.csv
처리된 row 수 : 194571, 순서 : 13, 파일명 : A000220.csv
처리된 row 수 : 194571, 순서 : 14, 파일명 : A000230.csv
처리된 row 수 : 194571, 순서 : 15, 파일명 : A000240.csv
처리된 row 수 : 194571, 순서 : 16, 파일명 : A000250.csv
처리된 row 수 : 194571, 순서 : 17, 파일명 : A000270.csv
처리된 row 수 : 194571, 순서 : 18, 파일명 : A000300.csv
처리된 row 수 : 194568, 순서 : 19, 파일명 : A000320.csv
처리된 row 수 : 194571, 순서 : 20, 파일명 : A000370.csv
처리된 row 수 : 194571, 순서 : 21, 파일명 : A000390.csv
처리된 row 수 : 194571, 순서

In [5]:
from module.setting import instCpCodeMgr, instStockChart, instCpCybos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from datetime import timedelta
from random import uniform
from tqdm import tqdm
import pandas as pd
import numpy as np
import pymysql
import csv
import os
import time
import glob
import shutil
import numpy as np

today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today() - timedelta(1)).day)

def get_pymysql_connection(ip_address, db_name):

    conn = pymysql.connect(host=ip_address, user='admin', password='big15'
                        , db=db_name, charset='utf8')

    return conn
    
def get_pymysql_stock_list(conn, db_name):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{0}'".format(db_name)

    with conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            result = [item[0] for item in cur.fetchall()]
            cur.close()

            return result

def get_krx_stock_list():

    target = r"D:/systrader-dev/data/kos*.csv"
    csv_list = glob.glob(target)

    # today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today()).day)

    if len(csv_list) >= 1:
        
        csv_date = csv_list[0].split('.')[0][-8:]
        
        if csv_date != today:

            os.remove(csv_list[0])
            os.remove(csv_list[1])
        else:
            return
        
    driver = webdriver.Chrome("./chromedriver")

    driver.get("http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201")
    driver.implicitly_wait(20)
    time.sleep(4)
    driver.find_element(By.XPATH, '//*[@id="MDCSTAT019_FORM"]/div[1]/div/table/tbody/tr/td/label[2]').click()
    time.sleep(1.5)
    driver.implicitly_wait(20)
    driver.find_element(By.XPATH, '//*[@id="MDCSTAT019_FORM"]/div[2]/div[1]/p[2]/button[2]').click()	
    time.sleep(1.5)
    driver.implicitly_wait(20)
    driver.find_element(By.XPATH, '//*[@id="ui-id-1"]/div/div[2]').click()
    time.sleep(1.5)
    driver.implicitly_wait(20)
    driver.find_element(By.XPATH, '//*[@id="MDCSTAT019_FORM"]/div[1]/div/table/tbody/tr/td/label[3]').click()
    time.sleep(1.5)
    driver.implicitly_wait(20)
    driver.find_element(By.XPATH, '//*[@id="MDCSTAT019_FORM"]/div[2]/div[1]/p[2]/button[2]').click()	
    time.sleep(1.5)
    driver.implicitly_wait(20)
    driver.find_element(By.XPATH, '//*[@id="ui-id-3"]/div/div[2]').click()
    time.sleep(1.5)
    driver.close()

    target = r"C:Users/TJ/Downloads/data*.csv"
    new_csv_list = glob.glob(target)

    file_size_1 = os.path.getsize(new_csv_list[0])
    file_size_2 = os.path.getsize(new_csv_list[1])

    if file_size_1 < file_size_2:
        shutil.move(new_csv_list[0], "../data/kospi_{0}.csv".format(today))
        shutil.move(new_csv_list[1], "../data/kosdaq_{0}.csv".format(today))
    else:
        shutil.move(new_csv_list[1], "../data/kospi_{0}.csv".format(today))
        shutil.move(new_csv_list[0], "../data/kosdaq_{0}.csv".format(today))

def get_compare_list(stock_list):
# stock code가 테이블이 존재하는 비교

    # today = str(datetime.today().year) + str(datetime.today().month) + str(datetime.today().day)

    kospi = pd.read_csv(f'../data/kospi_{today}.csv', encoding='euc-kr')
    kosdaq = pd.read_csv(f'../data/kosdaq_{today}.csv', encoding='euc-kr')

    kospi = kospi[kospi['주식종류'] == '보통주'].iloc[:,[1,-3]].iloc[:,0]
    kosdaq = kosdaq[kosdaq['주식종류'] == '보통주'].iloc[:,[1,-3]].iloc[:,0]

    kospi_kosdaq_list = pd.concat([kospi,kosdaq]).to_list()

    exist_list = []
    empty_list = []

    for code in kospi_kosdaq_list:
        if code in stock_list:
            exist_list.append(code)
        elif code not in stock_list:
            empty_list.append(code)
    
    exist_list.sort()
    empty_list.sort()
    
    return exist_list, empty_list

def update_pymysql_exist(code, type, conn):
# 존재할 경우, 대신증권에서 해당 종목 일봉 분봉 업데이트문


    sql = "SELECT MAX(날짜) FROM STOCK_INFO.{0} ".format(code)

    cur = conn.cursor()
    cur.execute(sql)
    max_day = cur.fetchone()[0] # 마지막 업데이트 날짜

    # sql = "SELECT MAX(시간) FROM STOCK_INFO.{0} WHERE 날짜={1}".format(code, today)

    # cur = conn.cursor()
    # cur.execute(sql)
    # max_time = cur.fetchone()[0] # 마지막 업데이트 날짜

    # today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today()).day)

    day_format = '%Y%m%d'
    start_day = datetime.strftime(datetime.strptime(str(max_day), day_format) + timedelta(days=1), day_format)

    # 최종 업데이트 날짜가 오늘 날짜와 같은 경우 배제
    if (str(max_day) != today):
        temp_df = pd.DataFrame()
        while True:
            instStockChart.SetInputValue(0, 'A'+code) # 종목명
            instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
            instStockChart.SetInputValue(3, start_day) # 요청 시작일 22일
            instStockChart.SetInputValue(2, start_day) # 요청 종료일 22일
            instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,18, 19,20,21,22,23,24,25,26])
            instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
            instStockChart.SetInputValue(9, ord('1'))

            instStockChart.BlockRequest() # 위 정보로 요청

            numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)


            index = []
            for i in range(numrow):
                index_ = str(instStockChart.GetDataValue(0,i))
                index.append(index_)

            
            stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

            for num in range(numrow):
                for col in range(len(numcolumn)):
                    # 1,2,3,4,5,6,7,8,9, 10
                    stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
                
            temp_df = pd.concat([stock_info, temp_df])
           

            start_day = datetime.strftime(datetime.strptime(start_day, day_format) + timedelta(days=1), day_format)
            transfer_end_day = datetime.strftime(datetime.strptime(today, day_format), day_format)
            time.sleep(uniform(0.15, 0.3))

            if instCpCybos.GetLimitRemainCount(1) < 3:
                time.sleep(10)

            if start_day > transfer_end_day:
                temp_df = temp_df.reset_index().rename(columns={'index':'날짜'})
                return temp_df
    else:
        return pd.DataFrame()

def update_pymysql_empty(code, type, conn):
# 존재할 경우, 대신증권에서 해당 종목 일봉 분봉 업데이트문
    
    day_format = '%Y%m%d'
    minus_day = timedelta(days=1)

    temp_df = pd.DataFrame()

    # today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today()).day)

    for day in range(730):
        instStockChart.SetInputValue(0, 'A'+code) # 종목명
        instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
        instStockChart.SetInputValue(3, today) # 요청 시작일 22일
        instStockChart.SetInputValue(2, today) # 요청 종료일 22일
        instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,18, 19,20,21,22,23,24,25,26])
        instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
        instStockChart.SetInputValue(9, ord('1'))

        instStockChart.BlockRequest() # 위 정보로 요청

        numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)


        index = []
        for i in range(numrow):
            index_ = str(instStockChart.GetDataValue(0,i))
            index.append(index_)

    
        stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

        for num in range(numrow):
            for col in range(len(numcolumn)):
                # 1,2,3,4,5,6,7,8,9, 10
                stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
        temp_df = pd.concat([stock_info, temp_df])

        today = datetime.strftime(datetime.strptime(today, day_format) - timedelta(days=1), day_format)
        transfer_end_day = datetime.strftime(datetime.strptime(today, day_format), day_format)
        time.sleep(uniform(0.15, 0.3))

        if instCpCybos.GetLimitRemainCount(1) < 3:
            time.sleep(10)
            
        temp_df = temp_df.reset_index().rename(columns={'index':'날짜'})
        return temp_df

# 삼성전자 종목 업데이트 우선 수행 후 template 생성

def save_samsung_template(conn):

    # today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today() - timedelta(1)).day)
    day_format = '%Y%m%d'
    yesterday = datetime.strftime(datetime.strptime(today, day_format) - timedelta(days=1), day_format)
    template = update_pymysql_exist('005930', 'm', conn)
    template = template.reset_index()
    print(template)
    template = template.set_index(['날짜', '시간'])
    template = template.notnull().replace(True, np.NaN)
    y_template = pd.read_csv('../data/{}_fillrow_template.csv'.format(yesterday), encoding='utf-8 sig')
    y_template = y_template.rename(columns={'Unnamed: 0' : '날짜'})
    y_template = y_template.set_index(['날짜', '시간'])
    template = pd.concat([template, y_template], axis=0, ignore_index=False)
    template.to_csv('../data/{}_fillrow_template.csv'.format(today), encoding='utf-8 sig')
    template = template.drop(['index'], axis=1)

    return template


# 종목별 업데이트, fillrow, concat 수행

def update_fillrow(min_df, template):
    
    min_df = min_df.set_index(['날짜','시간'])
    min_df = pd.merge(template, min_df, how='left', left_index=True, right_index=True)
    min_df = min_df.drop(columns=min_df.columns[0:24], axis=1)
    columns_temp = min_df.columns
    bin_columns = []
    for column in columns_temp:
        bin_columns.append(column.split('_')[0])

    min_df.columns = bin_columns

    min_df = min_df.reset_index()


    min_df[['거래량','거래대금']] = min_df[['거래량','거래대금']].fillna(0)
    min_df = min_df.bfill()
    min_df = min_df.dropna()
    min_df.to_csv('./merge.csv', encoding='utf-8-sig')
    return min_df

# 분봉 일봉 concate 진행

def update_concat(min_df, day_df):

    min_drop_list = ['전일대비','상장주식수','시가총액','외국인주문한도수량'
                ,'외국인주문가능수량','외국인현보유수량','외국인현보유비율'
                ,'수정주가일자','수정주가비율','기관순매수량','기관누적순매수량'
                ,'등락주선','등락비율','예탁금','주식회전율','거래성립률']                                  
    day_drop_list = ['시간','시가','고가','저가','종가','거래량','거래대금','누적체결매도수량', '누적체결매수수량'
                ,'등락주선','등락비율','예탁금','주식회전율','거래성립률']
        
    stock_min_df = min_df.drop(min_drop_list, axis=1)
    stock_day_df = day_df.drop(day_drop_list, axis=1)
    
    concat_df = pd.merge(stock_min_df, stock_day_df, on='날짜', how='left')

    return concat_df

# label 추가

def update_label(concat_df):
    
    # 데이터 프레임에서 날짜 인덱스 추출
    concat_df.set_index('날짜', inplace=True)
    date = concat_df.index.unique()
    # label 컬럼 추가 후 0으로 초기화
    concat_df['label'] = 0
    # 업데이트에 사용할 데이터 프레임 생성
    update_stock_info = pd.DataFrame()
    # labeling
    for day in date:
        # 특정일의 Data 추출
        select_day = concat_df.loc[day].copy()
        select_day['label'] = 0
        # 특정일의 Row 만큼 반복
        for row in range(len(select_day)):
            # 특정일의 현재 row 이후 최대 고가를 추출
            next_price = select_day[-row-1::-1]['고가'].max()
            # 추출한 최대 고가를 label 컬럼에 대입
            select_day.iloc[-row-1,-1] = next_price
            next_price = 0
        # 특정일 label이 추가된 DF를 업데이트할 DF에 concat
        update_stock_info = pd.concat([update_stock_info, select_day])

    trans = update_stock_info.loc[:,['고가','label']]
    trans['고가'] = trans['고가'].astype('float')
    trans['label'] = trans['label'].astype('float')
    trans = trans.rename(columns={'label':'pct_label'}).T

    get_trans = trans.pct_change().T.iloc[:,-1]
    update_stock_info = pd.concat([update_stock_info, get_trans], axis=1)
    update_stock_info = update_stock_info.drop(['label'], axis=1)

    update_stock_info['pct_label'] = update_stock_info['pct_label'].mul(100)
    update_stock_info['pct_label'] = update_stock_info['pct_label'].round(1)
    update_stock_info.reset_index(inplace=True)

    return update_stock_info


# Data csv => SQL에 추가
def sqlalchemy_update_insert(update_df, code, type, conn):

    update_df['년'] = update_df['날짜'].apply(str).str[:4]
    update_df['월'] = update_df['날짜'].apply(str).str[4:6]
    update_df['일'] = update_df['날짜'].apply(str).str[6:8]
    update_df.sort_index(ascending=False, inplace=True)
    # name= table명
    # con= connect할 때 들어가는 아이디 비밀번호 등등
    # if_exists= 'append' : 기존 테이블이 있는 경우 데이터를 추가
    # 'fail' : 기존 테이블이 있는 경우 아무일도 없지만 없을 경우 valuesError뜸
    # 'replace' : 기존 테이블이 있을 경우, 기존 테이블을 삭제하고 다시 테이블을
    count = update_df.to_sql(name='{0}'.format(code), con=conn, if_exists=type, index=False)
    
# for문 완료 후 연결 해제
    conn.close()

In [6]:

##########################################################################################################
pymysql_conn = get_pymysql_connection('192.168.50.123', 'stock_info')

stock_list = get_pymysql_stock_list(pymysql_conn, 'stock_info')
get_krx_stock_list()
exist_list, empty_list = get_compare_list(stock_list)
pymysql_conn = get_pymysql_connection('192.168.50.123', 'stock_info')
template = save_samsung_template(pymysql_conn)
for code in exist_list:
    sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'stock_info')
    pymysql_conn = get_pymysql_connection('192.168.50.123', 'stock_info')
    print('현재 업데이트 중인 종목명 : ' + code)
    min_df = update_pymysql_exist(code, 'm', pymysql_conn)
    if min_df.empty == False:
        day_df = update_pymysql_exist(code, 'D', pymysql_conn)
        min_df = update_fillrow(min_df, template)
        concat_df = update_concat(min_df, day_df)
        update_df = update_label(concat_df)
        sqlalchemy_update_insert(update_df, code, 'append', sqlalchemy_conn)


# for code in empty_list:
#     sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'stock_info')
#     pymysql_conn = get_pymysql_connection('192.168.50.123', 'stock_info')

#     min_df = update_pymysql_empty(code, 'm', pymysql_conn)
#     day_df = update_pymysql_empty(code, 'D', pymysql_conn)

#     min_df = update_fillrow(min_df)
#     concat_df = update_concat(min_df, day_df)
#     update_df = update_label(concat_df)
#     sqlalchemy_update_insert(update_df, code, 'replace', sqlalchemy_conn)
    

      index        날짜    시간     시가     고가     저가     종가 전일대비     거래량  \
0         0  20221128  1530  60100  60100  60100  60100    0  840274   
1         1  20221128  1520  60000  60100  60000  60100    0   31833   
2         2  20221128  1519  60000  60100  60000  60000    0   15740   
3         3  20221128  1518  60000  60100  60000  60000    0   20547   
4         4  20221128  1517  60000  60100  60000  60000    0   22464   
...     ...       ...   ...    ...    ...    ...    ...  ...     ...   
1900   1900  20221122   905  61000  61000  60800  61000    0  232338   
1901   1901  20221122   904  61000  61000  60900  61000    0   15629   
1902   1902  20221122   903  60800  61000  60800  60900    0   92486   
1903   1903  20221122   902  60900  60900  60700  60800    0  253103   
1904   1904  20221122   901  60900  61000  60800  60900    0  146274   

             거래대금  ... 외국인현보유비율 수정주가일자 수정주가비율 기관순매수량 기관누적순매수량 등락주선 등락비율 예탁금  \
0     50500460000  ...      0.0      0    0.0      0    

In [93]:
template

NameError: name 'template' is not defined

In [ ]:
host_name = "localhost"
username = 'root'
password = 'big15'
database_name = 'big15'

In [ ]:
db1 = pymysql.connect(
    host= host_name
    , port=3306
    , user=username
    , password=password
    , db=database_name
    , charset='utf8'
)

In [ ]:
sql = 'select * from stock'
df2 = pd.read_sql(sql,con=db1)
print(df2)

c:\Users\TJ\.conda\envs\tf2\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


        Unnamed: 0    시간     시가     고가     저가     종가  전일대비     거래량  \
0         20221103  1530  82700  82700  82700  82700     0  312503   
1         20221103  1520  83100  83100  83000  83100     0   12246   
2         20221103  1519  83000  83100  83000  83100     0   19656   
3         20221103  1518  83200  83200  83000  83000     0   39559   
4         20221103  1517  83100  83200  83100  83200     0    4789   
...            ...   ...    ...    ...    ...    ...   ...     ...   
190375    20201027   905  82000  82300  81800  82200     0   53979   
190376    20201027   904  82200  82300  82000  82000     0   19926   
190377    20201027   903  82200  82300  82100  82200     0   16356   
190378    20201027   902  82500  82500  82200  82200     0   12931   
190379    20201027   901  82300  82600  82200  82600     0   41258   

               거래대금  누적체결매도수량  ...  외국인현보유비율  수정주가일자  수정주가비율  기관순매수량  \
0       25843990000   1314512  ...       0.0       0     0.0       0   
1        101726

In [ ]:
df2 = df2.astype({'Unnamed: 0':'int32'})
df2 = df2.astype({'시간':'int16'})
df2 = df2.astype({'시가':'int32'})
df2 = df2.astype({'고가':'int32'})
df2 = df2.astype({'저가':'int32'})
df2 = df2.astype({'종가':'int32'})
df2 = df2.astype({'전일대비':'int32'})
df2 = df2.astype({'거래량':'int32'})
df2 = df2.astype({'거래대금':'int64'})
df2 = df2.astype({'누적체결매도수량':'int32'})
df2 = df2.astype({'상장주식수':'int32'})
df2 = df2.astype({'시가총액':'int64'})
df2 = df2.astype({'외국인주문한도수량':'int32'})
df2 = df2.astype({'외국인주문가능수량':'int32'})
df2 = df2.astype({'외국인현보유수량':'int32'})
df2 = df2.astype({'외국인현보유비율':'float32'})
df2 = df2.astype({'수정주가일자':'int32'})
df2 = df2.astype({'수정주가비율':'float32'})
df2 = df2.astype({'기관순매수량':'int32'})
df2 = df2.astype({'기관누적순매수량':'int32'})
df2 = df2.astype({'등락주선':'int32'})
df2 = df2.astype({'등락비율':'float32'})
df2 = df2.astype({'예탁금':'int64'})
df2 = df2.astype({'주식회전율':'float32'})
df2 = df2.astype({'거래성립률':'float32'})


In [ ]:
sql = 'SELECT COUNT(*) FROM STOCK'
print(pd.read_sql(sql=sql, con=db1))

   COUNT(*)
0    190380


c:\Users\TJ\.conda\envs\tf2\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
pymysql_conn = get_pymysql_connection('192.168.50.123', 'stock_info')

stock_list = get_pymysql_stock_list(pymysql_conn, 'stock_info')
get_krx_stock_list()
exist_list, empty_list = get_compare_list(stock_list)

date = [20221122, 20221123, 20221124, 20221125, 20221128 , 20221129]

def del_sql(code, date):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    for day in date:
        conn = get_pymysql_connection('192.168.50.123', 'stock_info')
        sql = "DELETE FROM stock_info.{0} WHERE 날짜='{1}'".format(code, day)

        cur = conn.cursor()
        print(sql)
        result = cur.execute(sql)
        print(result)
        conn.commit()


for code in exist_list:
    del_sql(code, date)
    


DELETE FROM stock_info.000020 WHERE 날짜='20221122'
381
DELETE FROM stock_info.000020 WHERE 날짜='20221123'
381
DELETE FROM stock_info.000020 WHERE 날짜='20221124'
381
DELETE FROM stock_info.000020 WHERE 날짜='20221125'
381
DELETE FROM stock_info.000020 WHERE 날짜='20221128'
0
DELETE FROM stock_info.000020 WHERE 날짜='20221129'
0
DELETE FROM stock_info.000040 WHERE 날짜='20221122'
381
DELETE FROM stock_info.000040 WHERE 날짜='20221123'
381
DELETE FROM stock_info.000040 WHERE 날짜='20221124'
381
DELETE FROM stock_info.000040 WHERE 날짜='20221125'
381
DELETE FROM stock_info.000040 WHERE 날짜='20221128'
381
DELETE FROM stock_info.000040 WHERE 날짜='20221129'
0
DELETE FROM stock_info.000050 WHERE 날짜='20221122'
381
DELETE FROM stock_info.000050 WHERE 날짜='20221123'
381
DELETE FROM stock_info.000050 WHERE 날짜='20221124'
381
DELETE FROM stock_info.000050 WHERE 날짜='20221125'
381
DELETE FROM stock_info.000050 WHERE 날짜='20221128'
381
DELETE FROM stock_info.000050 WHERE 날짜='20221129'
0
DELETE FROM stock_info.000060 WHERE 

In [14]:
import csv
pymysql_conn = get_pymysql_connection('192.168.50.123', 'stock_info')

stock_list = get_pymysql_stock_list(pymysql_conn, 'stock_info')
get_krx_stock_list()
exist_list, empty_list = get_compare_list(stock_list)

def del_sql(code):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    conn = get_pymysql_connection('192.168.50.123', 'stock_info')
    sql = "ALTER TABLE stock_info.{0} ADD PRIMARY KEY (날짜, 시간)".format(code)

    cur = conn.cursor()
    print(sql)
    result = cur.execute(sql)
    print(result)
    conn.commit()


error_list = []

for code in exist_list[0:2]:
    try:
        del_sql(code)
    except:
        print('예외 발생 종목 : ' + code)
        error_list.append(code)

print(error_list)
    


ALTER TABLE stock_info.000020 ADD PRIMARY KEY (날짜, 시간)
